# Catchment area analysis of supermarkets

## Libraries and settings

In [1]:
# Libraries
import os
import time
import glob
import json
import folium
import requests
import numpy as np
import pandas as pd
import geopandas as gdp
import matplotlib.pyplot as plt
from openrouteservice import client
from IPython.display import clear_output

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Import supermarket data

In [2]:
# Read OpenRouteService API key
with open(file='ORS_TOKEN_README.txt', mode='r') as file:
    api_key = file.read()
# print(api_key)

# Import supermarket data
df_orig = pd.read_csv('supermarkets_data_enriched.csv', 
                      sep=',',
                      encoding='utf-8')[['id',
                                         'lat',
                                         'lon',
                                         'brand',
                                         'bfs_number',
                                         'bfs_name',
                                         'addr:housenumber',
                                         'addr:postcode',
                                         'geometry']]

# Subset, only supermarkets with complete address
df = df_orig.dropna()
print(df.shape)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'ors_token.txt'

## Create isochrone for a single supermarket

In [ ]:
# Settings
municip = 'Adliswil'
transport = 'driving-car' # alternativle 'foot-walking'
traveltime = 15

# Client settings
ors = client.Client(key=api_key)

# Set up supermarket dictionary with a single supermarket
df_sub = df.loc[df['bfs_name'] == municip].iloc[0]
supermarkets = {df_sub['brand']: {'location': [df_sub['lon'], 
                                               df_sub['lat']]}}
print(supermarkets)

# Set up folium map
map = folium.Map(tiles='Stamen Toner', 
                 location=([df_sub['lat'], 
                            df_sub['lon']]), 
                 zoom_start=11)

# Parameters for server-request
params_iso = {'profile': transport,
              'range': [traveltime*60],
              'attributes': ['total_pop']}

# Server request
for name, apt in supermarkets.items():
    
    # Add coords to request parameters
    params_iso['locations'] = [apt['location']]
    
    # Perform isochrone request
    apt['iso'] = ors.isochrones(**params_iso)
    
    # Add GeoJson to map
    folium.features.GeoJson(apt['iso']).add_to(map)
    
    # Save GeoJson as file
    # with open(f'{name}.json', 'w') as f:
        # f.write(json.dumps(apt['iso']))

    # Reverse coords due to weird folium lat/lon syntax
    folium.map.Marker(list(reversed(apt['location'])),
                      icon = folium.Icon(color='green',
                                         icon_color='#cc0000',
                                         icon='home',
                                         prefix="fa"),
                      popup = name
                      ).add_to(map)

# Plot map
map

## Getting the number of residents in the isochrone area

In [ ]:
# Save the data as pandas data frame
data = pd.DataFrame(apt['iso']["features"])

# Create data frame from column 'properties'
df_pop = pd.DataFrame(data.loc[0, 'properties'])
val = df_pop['total_pop'][0]
print('Number of residents in isochrone area:', f'{val:,.0f}')

## Estimating the purchasing power in the isochrone area

In [ ]:
# Simple assumtions:
# 500 CHF per resident and month
# Residents / employee ratio = 2:1
# 20 CHF per employee and working day

# Result
val = ((df_pop['total_pop'][0] * 500) + (df_pop['total_pop'][0] * 0.5 * 20 * 20)) / 10**6
print('Purchasing power per month:',
      f'{val:.1f}',
     'Mio. CHF')

## Automization of the analysis for multiple of supermarkets

In [ ]:
# Client settings
ors = client.Client(key=api_key)

# Parameters for server-request
params_iso = {'profile': 'driving-car',
              'range': [15*60],
              'attributes': ['total_pop']}

# Df to store the data
pop_out = []

try:
    for i in range(0, 10):
        
        print('Preparing isochrone for supermarkt', df[['id', 
                                                        'brand', 
                                                        'bfs_name', 
                                                        'addr:housenumber',
                                                        'addr:postcode']].iloc[i])
        
        supermarkets = {df['brand'].iloc[i]: {'location': [df['lon'].iloc[i],
                                                           df['lat'].iloc[i]]}}
        # Server request
        for name, apt in supermarkets.items():
            
            # Add coords to request parameters
            params_iso['locations'] = [apt['location']]
            
            # Perform isochrone request
            apt['iso'] = ors.isochrones(**params_iso)
            time.sleep(1)
            
            # Save the data as pandas data frame
            data = pd.DataFrame(apt['iso']["features"])
            
            # Create data frame from column 'properties'
            df_pop = pd.DataFrame(data.loc[0, 'properties'])
            
            # Write values
            pop_out.append([df['id'].iloc[i], 
                            df['brand'].iloc[i],
                            df['bfs_number'].iloc[i], 
                            df['bfs_name'].iloc[i], 
                            df['addr:housenumber'].iloc[i],
                            df['addr:postcode'].iloc[i],
                            df_pop['total_pop'][0]])
            clear_output(wait=True)
            
except:
    pop_out.append([df['id'].iloc[i], 
                    df['brand'].iloc[i],
                    df['bfs_number'].iloc[i], 
                    df['bfs_name'].iloc[i], 
                    df['addr:housenumber'].iloc[i],
                    df['addr:postcode'].iloc[i],
                    np.NaN])
    clear_output(wait=True)

# Print list
pop_out

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')